In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score
from catboost import CatBoostClassifier

# 데이터 로드
data = pd.read_csv("../monunmon.csv")

# Label 값에 1을 더하여 0부터 시작하는 연속된 정수로 변환
data['Label'] = data['Label'] + 1

# 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

# Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

### RandomizedSearchCV를 이용한 하이퍼파라미터 튜닝 ###
param_dist = {
    'depth': [4, 6, 8, 10],            # 트리 깊이
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # 학습률
    'iterations': [500, 1000, 1500],   # 반복 횟수
    'l2_leaf_reg': [1, 3, 5, 7],       # L2 정규화
    'bagging_temperature': [0.1, 0.5, 1], # 배깅 온도
    'border_count': [32, 64, 128]      # 이진 분할 개수
}

random_search = RandomizedSearchCV(
    estimator=CatBoostClassifier(
        random_seed=42,
        loss_function='MultiClass',
        eval_metric='MultiClass',
        verbose=0
    ),
    param_distributions=param_dist,
    n_iter=50,                         # 랜덤 샘플링 횟수
    scoring='f1_weighted',             # 평가 기준
    cv=3,                              # 3-겹 교차 검증
    verbose=2,                         # RandomizedSearchCV 진행 상황 출력
    n_jobs=-1,                         # 병렬 처리
    random_state=42
)

# RandomizedSearchCV 실행
print("Starting RandomizedSearchCV...")
random_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation Score:", random_search.best_score_)

# 최적 모델로 테스트 데이터 평가
best_model = random_search.best_estimator_
y_test_pred = best_model.predict(X_test)

# 최적 모델 성능 출력
print("\nClassification Report on Test Data (Tuned CatBoost):")
print(classification_report(y_test, y_test_pred))

test_accuracy_tuned = accuracy_score(y_test, y_test_pred)
test_f1_tuned = f1_score(y_test, y_test_pred, average='weighted')

print(f"Test Accuracy (Tuned): {test_accuracy_tuned}")
print(f"Test F1 Score (Tuned): {test_f1_tuned}")


Fitting 3 folds for each of 24 candidates, totalling 72 fits
